In [92]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.sql import Window
import pyspark.sql.types as T
from pyspark.sql.functions import col, rank

spark = SparkSession.builder.getOrCreate()

In [95]:
df.count()

1338

#### difference of groupby and window

In [91]:
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 4000),
    ("Cathy", "HR", 3500),
    ("David", "HR", 4500),
    ("Eve", "IT", 6000),
    ("Frank", "IT", 7000),
]
columns = ["Name", "Department", "Salary"]

df = spark.createDataFrame(data, columns)
df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Alice|     Sales|  5000|
|  Bob|     Sales|  4000|
|Cathy|        HR|  3500|
|David|        HR|  4500|
|  Eve|        IT|  6000|
|Frank|        IT|  7000|
+-----+----------+------+



In [103]:
window_spec = Window.partitionBy("Department").orderBy("Salary")

row = df.withColumn("RowNumber", row_number().over(window_spec))
row.show()

+-----+----------+------+---------+
| Name|Department|Salary|RowNumber|
+-----+----------+------+---------+
|Cathy|        HR|  3500|        1|
|David|        HR|  4500|        2|
|  Eve|        IT|  6000|        1|
|Frank|        IT|  7000|        2|
|  Bob|     Sales|  4000|        1|
|Alice|     Sales|  5000|        2|
+-----+----------+------+---------+



In [104]:
df.groupBy("Department").agg(F.avg('Salary').alias('Average Salary')).show()

+----------+--------------+
|Department|Average Salary|
+----------+--------------+
|     Sales|        4500.0|
|        HR|        4000.0|
|        IT|        6500.0|
+----------+--------------+



#### windows operations

In [105]:
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())

data = df.withColumn("row_no", row_number().over(window_spec))
data.show()

+-----+----------+------+------+
| Name|Department|Salary|row_no|
+-----+----------+------+------+
|David|        HR|  4500|     1|
|Cathy|        HR|  3500|     2|
|Frank|        IT|  7000|     1|
|  Eve|        IT|  6000|     2|
|Alice|     Sales|  5000|     1|
|  Bob|     Sales|  4000|     2|
+-----+----------+------+------+



In [106]:
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())

data1 = df.withColumn("rank", rank().over(window_spec))
data1.show()

+-----+----------+------+----+
| Name|Department|Salary|rank|
+-----+----------+------+----+
|David|        HR|  4500|   1|
|Cathy|        HR|  3500|   2|
|Frank|        IT|  7000|   1|
|  Eve|        IT|  6000|   2|
|Alice|     Sales|  5000|   1|
|  Bob|     Sales|  4000|   2|
+-----+----------+------+----+



In [107]:
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())

data3 = df.withColumn("Dens_rank", dense_rank().over(window_spec))
data3.show()

+-----+----------+------+---------+
| Name|Department|Salary|Dens_rank|
+-----+----------+------+---------+
|David|        HR|  4500|        1|
|Cathy|        HR|  3500|        2|
|Frank|        IT|  7000|        1|
|  Eve|        IT|  6000|        2|
|Alice|     Sales|  5000|        1|
|  Bob|     Sales|  4000|        2|
+-----+----------+------+---------+



In [109]:
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())

data4 = df.withColumn("previous_Salary", lag("Salary", 1).over(window_spec))
data4.show()

+-----+----------+------+---------------+
| Name|Department|Salary|previous_Salary|
+-----+----------+------+---------------+
|David|        HR|  4500|           NULL|
|Cathy|        HR|  3500|           4500|
|Frank|        IT|  7000|           NULL|
|  Eve|        IT|  6000|           7000|
|Alice|     Sales|  5000|           NULL|
|  Bob|     Sales|  4000|           5000|
+-----+----------+------+---------------+



In [110]:
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())

data5 = df.withColumn("next_Salary", lag("Salary", 1).over(window_spec))
data5.show()

+-----+----------+------+-----------+
| Name|Department|Salary|next_Salary|
+-----+----------+------+-----------+
|David|        HR|  4500|       NULL|
|Cathy|        HR|  3500|       4500|
|Frank|        IT|  7000|       NULL|
|  Eve|        IT|  6000|       7000|
|Alice|     Sales|  5000|       NULL|
|  Bob|     Sales|  4000|       5000|
+-----+----------+------+-----------+



#### MissingValues

In [111]:
df.filter(df.Salary.isNotNull()).show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Alice|     Sales|  5000|
|  Bob|     Sales|  4000|
|Cathy|        HR|  3500|
|David|        HR|  4500|
|  Eve|        IT|  6000|
|Frank|        IT|  7000|
+-----+----------+------+



In [112]:
df.filter(df.Salary.isNull()).show()

+----+----------+------+
|Name|Department|Salary|
+----+----------+------+
+----+----------+------+



In [113]:
df.na.drop().show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Alice|     Sales|  5000|
|  Bob|     Sales|  4000|
|Cathy|        HR|  3500|
|David|        HR|  4500|
|  Eve|        IT|  6000|
|Frank|        IT|  7000|
+-----+----------+------+



#### DF operations

In [114]:
df.filter(df["Department"].isin("Sales")).show()



+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|Alice|     Sales|  5000|
|  Bob|     Sales|  4000|
+-----+----------+------+



In [115]:
df.filter(df["Name"].like("D%")).show()


+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
|David|        HR|  4500|
+-----+----------+------+



In [116]:
df.withColumn("first_3_letter",df["Name"].substr(1,3)).show()

+-----+----------+------+--------------+
| Name|Department|Salary|first_3_letter|
+-----+----------+------+--------------+
|Alice|     Sales|  5000|           Ali|
|  Bob|     Sales|  4000|           Bob|
|Cathy|        HR|  3500|           Cat|
|David|        HR|  4500|           Dav|
|  Eve|        IT|  6000|           Eve|
|Frank|        IT|  7000|           Fra|
+-----+----------+------+--------------+



In [117]:
df = spark.read.csv(r"C:\Users\ranju\Downloads\insurance.csv",inferSchema= True,header=True)
df.show()

+---+------+------+--------+------+---------+-----------+
|age|gender|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

#### Rank Individuals by Charges Within Each Region


In [74]:

windowSpec = Window.partitionBy("region").orderBy(col("charges").desc())

rank = df.withColumn("Rank", rank().over(windowSpec))

rank.show()


+---+------+------+--------+------+---------+-----------+----+
|age|gender|   bmi|children|smoker|   region|    charges|Rank|
+---+------+------+--------+------+---------+-----------+----+
| 31|female|38.095|       1|   yes|northeast|58571.07448|   1|
| 54|  male|40.565|       3|   yes|northeast|48549.17835|   2|
| 61|female|36.385|       1|   yes|northeast|48517.56315|   3|
| 59|female|36.765|       1|   yes|northeast|47896.79135|   4|
| 51|female| 37.05|       3|   yes|northeast| 46255.1125|   5|
| 62|  male|32.015|       0|   yes|northeast|45710.20785|   6|
| 51|female| 34.96|       2|   yes|northeast| 44641.1974|   7|
| 36|  male|41.895|       3|   yes|northeast|43753.33705|   8|
| 53|  male|34.105|       0|   yes|northeast|43254.41795|   9|
| 55|  male|30.685|       0|   yes|northeast|42303.69215|  10|
| 46|female| 35.53|       0|   yes|northeast| 42111.6647|  11|
| 50|  male|  32.3|       1|   yes|northeast|  41919.097|  12|
| 50|  male|31.825|       0|   yes|northeast|41097.1617

#### Calculate the Average BMI for Each Gender


In [75]:
windowSpec = Window.partitionBy("gender")

avg = df.withColumn("average_bmi",F. avg("bmi").over(windowSpec))
avg.show()


+---+------+------+--------+------+---------+-----------+------------------+
|age|gender|   bmi|children|smoker|   region|    charges|       average_bmi|
+---+------+------+--------+------+---------+-----------+------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|30.377749244713023|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|30.377749244713023|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|30.377749244713023|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|30.377749244713023|
| 60|female| 25.84|       0|    no|northwest|28923.13692|30.377749244713023|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|30.377749244713023|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|30.377749244713023|
| 52|female| 30.78|       1|    no|northeast| 10797.3362|30.377749244713023|
| 60|female|36.005|       0|    no|northeast|13228.84695|30.377749244713023|
| 30|female|  32.4|       1|    no|southwest|   4149.736|30.377749244713023|

#### Row Number for Each Individual in Their Region


In [76]:
window_spec = Window.partitionBy("region").orderBy("age")
row = df.withColumn("row_number",row_number().over(window_spec))
row.show()

+---+------+------+--------+------+---------+-----------+----------+
|age|gender|   bmi|children|smoker|   region|    charges|row_number|
+---+------+------+--------+------+---------+-----------+----------+
| 18|female|26.315|       0|    no|northeast| 2198.18985|         1|
| 18|female|38.665|       2|    no|northeast| 3393.35635|         2|
| 18|female|35.625|       0|    no|northeast| 2211.13075|         3|
| 18|female|30.115|       0|    no|northeast| 21344.8467|         4|
| 18|  male| 23.75|       0|    no|northeast|  1705.6245|         5|
| 18|  male|25.175|       0|   yes|northeast|15518.18025|         6|
| 18|  male| 15.96|       0|    no|northeast|  1694.7964|         7|
| 18|  male| 17.29|       2|   yes|northeast| 12829.4551|         8|
| 18|  male| 22.99|       0|    no|northeast|  1704.5681|         9|
| 18|  male|  30.4|       3|    no|northeast|   3481.868|        10|
| 18|female|29.165|       0|    no|northeast|7323.734819|        11|
| 18|female|30.115|       0|    no

#### Find the Difference Between Charges and the Average Charges by Region

In [77]:
window_spec = Window.partitionBy("region")

avg_charge = df.withColumn("average_charge",F.avg("charges").over(window_spec))

diff = avg_charge.withColumn("difference",col("charges") - col("average_charge"))

diff.show()

+---+------+------+--------+------+---------+-----------+----------------+-------------------+
|age|gender|   bmi|children|smoker|   region|    charges|  average_charge|         difference|
+---+------+------+--------+------+---------+-----------+----------------+-------------------+
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|13406.3845163858|   -6999.9738163858|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|13406.3845163858|-10685.063716385801|
| 52|female| 30.78|       1|    no|northeast| 10797.3362|13406.3845163858|-2609.0483163858007|
| 23|  male|23.845|       0|    no|northeast| 2395.17155|13406.3845163858|  -11011.2129663858|
| 60|female|36.005|       0|    no|northeast|13228.84695|13406.3845163858|-177.53756638580126|
| 34|female| 31.92|       1|   yes|northeast| 37701.8768|13406.3845163858|   24295.4922836142|
| 63|female|23.085|       0|    no|northeast|14451.83515|13406.3845163858| 1045.4506336142003|
| 18|female|26.315|       0|    no|northeast| 2198